In [ ]:
%config Completer.use_jedi = False

# Read data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

print("movies samples: {}, features: {}".format(movies.shape[0], movies.shape[1]))
print("ratings shape: {}, features: {}".format(ratings.shape[0], ratings.shape[1]))

In [ ]:
movies.head(n=3)

In [ ]:
ratings.head(n=3)

# Pivot ratings -> userId - movieId

In [ ]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table(values='rating', index='userId', columns='movieId')
ratings_matrix.head()

In [ ]:
# Turn numbers into titles
ratings_movies = ratings.merge(movies, left_on='movieId', right_on='movieId')
ratings_matrix = ratings_movies.pivot_table(values='rating', index='userId', columns='title')

# Fill Nan to 0
ratings_matrix.fillna(0, inplace=True)
ratings_matrix.head()

# Cosine similarity matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Transpose ratings_matrix
ratings_matrix_T = ratings_matrix.T

# Calculate cosine similarity array
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# Change numpy array to pandas dataframe
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

In [ ]:
item_sim_df

## Personalized recommend system: based items k-nearest collaborative filtering

- hat_R_ui: 영화 'i'에 대한 사용자 'u'의 평점 예측값
- S_iN: 영화 'i'와 가장 유사도가 높은 top-N개 영화의 유사도 벡터
- R_uN: 사용자 u의 영화 i와 가장 유사도가 높은 top-N개 영황에 대한 실제 평점 벡터